In [1]:
import numpy as np

In [8]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
import gensim
from gensim.utils import simple_preprocess

In [6]:
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

In [10]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
!pip install bertopic

     |████████████████████████████████| 58 kB 3.3 MB/s 
     |████████████████████████████████| 5.2 MB 9.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 79 kB 8.6 MB/s 
     |████████████████████████████████| 88 kB 7.1 MB/s 
     |████████████████████████████████| 4.0 MB 46.1 MB/s 
     |████████████████████████████████| 1.2 MB 40.5 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 636 kB 71.0 MB/s 
     |████████████████████████████████| 880 kB 66.9 MB/s 
     |████████████████████████████████| 6.6 MB 52.9 MB/s 
     |████████████████████████████████| 1.1 MB 68.0 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2330798 sha256=026c5cfc077e7af1c5f648cd2c97c95eba02cfec9a5bb2578ba5c8505615597f
  Stored in directory: /root/.cache/pip/wheels/6e/7a/5e/259ccc841c085fc41b99ef4

In [ ]:
'''from sklearn.datasets import fetch_20newsgroups
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']'''

In [2]:

newsgroups_train = fetch_20newsgroups(subset='train', shuffle = True,remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', shuffle = True,remove=('headers', 'footers', 'quotes'))

In [52]:
from sklearn.datasets import fetch_20newsgroups
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

In [53]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True,nr_topics=20)
#cleaned_docs=topic_model._preprocess_text(docs)
topics, probs = topic_model.fit_transform(docs)


Batches:   0%|          | 0/589 [00:00<?, ?it/s]

2022-05-04 09:50:26,268 - BERTopic - Transformed documents to Embeddings
2022-05-04 09:50:47,068 - BERTopic - Reduced dimensionality
2022-05-04 09:51:20,546 - BERTopic - Clustered reduced embeddings
2022-05-04 09:51:31,550 - BERTopic - Reduced number of topics from 215 to 21


In [54]:
freq = topic_model.get_topic_info(); freq.head(-1)

,Topic,Count,Name
0,-1,11416,-1_the_to_of_and
1,0,1879,0_the_in_to_he
2,1,643,1_the_to_of_key
3,2,534,2_card_monitor_the_video
4,3,527,3_idjits_ites_cheek_dancing
5,4,487,4_the_of_israel_to
6,5,347,5_the_to_you_bike
7,6,338,6_drive_scsi_drives_disk
8,7,328,7_the_space_of_and
9,8,308,8_car_the_and_it


In [55]:
topic_model.get_topic(5)

[('the', 0.03450149878873217),
 ('to', 0.032612715392929995),
 ('you', 0.02862522657981487),
 ('bike', 0.027252796313778808),
 ('and', 0.026753694361563503),
 ('my', 0.02514100239323245),
 ('on', 0.023856001214961745),
 ('of', 0.022121637187892937),
 ('it', 0.021751044171193158),
 ('in', 0.020682266770520347)]

In [56]:
topic_model.visualize_topics()

In [57]:
topic_model.visualize_distribution(probs[103], min_probability=0.005)

In [58]:
topic_model.visualize_barchart()

In [59]:
cleaned_docs = topic_model._preprocess_text(docs)

In [60]:
vectorizer = topic_model.vectorizer_model
tokenizer = vectorizer.build_tokenizer()
tokens = [tokenizer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]
# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()

In [61]:
coherence

0.4465307448889145

In [62]:
coherence_model2 = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='u_mass')
coherence2= coherence_model2.get_coherence()

In [63]:
coherence2

-1.569404104233259

In [64]:
coherence_model3 = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_npmi')
coherence3= coherence_model3.get_coherence()

In [65]:
coherence3

0.006576237018709267